In [1]:
import os, random
from os import path
from numpy.random import seed
from keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot as plt
import numpy as np

from distutils.dir_util import copy_tree
import shutil

random.seed(42)

In [2]:
base_path=os.getcwd()


In [3]:
dir_path=(base_path+r"\tumor_data")

In [12]:
for index, letter in (enumerate(os.listdir(dir_path))):
    folder_letter = os.path.join(dir_path, letter)
    print(letter)

1
2
3
test
train
validation


In [13]:
# rename the images to 1_xx, 2_xx, 3_xx, by the labels 

for _, letter in enumerate(os.listdir(dir_path)):
    folder_letter = os.path.join(dir_path, letter)
 
    counts=0
    for _, image in enumerate(os.listdir(folder_letter)):
        
        dst=image
        #print(dst)
        if '_' in dst:
            continue
        else:
            dst = str(letter) + "_" + str(counts) + ".png"
            #print(dst)
            src = os.path.join(folder_letter, image)
            dst = os.path.join(folder_letter, dst)
            print(counts)
            os.rename(src, dst)
            
        counts=counts+1
        

In [5]:
# define directories for the train, val and test splits
train_path = os.path.join(dir_path, 'train')
val_path = os.path.join(dir_path, 'validation')
test_path = os.path.join(dir_path, 'test')

In [14]:


def createdir(mydir):
    try:
        os.mkdir(mydir)
    except OSError:
        pass

# create the directories

createdir(train_path)
createdir(val_path)
createdir(test_path)


In [15]:
# create the directories for each label for each train, validationa and test folders
count=0
for dir in [train_path, val_path, test_path]:
    count+=1
    for letter in ['1','2','3']:
        #print('hel')
        if count == 1:
            createdir(os.path.join(dir,str("train_{0}".format(str(letter)))))
        elif count == 2:
            createdir(os.path.join(dir,str("val_{0}".format(str(letter)))))
            
        elif count == 3:
            createdir(os.path.join(dir,str("test_{0}".format(str(letter)))))
        

In [16]:
# copy all the images for the train folder to then split them for validation and test folders 

count=0

for dir_ in [train_path+r'\train_1', train_path+r'\train_2', train_path+r'\train_3']:
        if len(os.listdir(dir_)) != 0:
            print('break')
            for f in os.listdir(dir_):
                os.remove(os.path.join(dir_, f))
for dir in [dir_path+r"\1", dir_path+r"\2", dir_path+r"\3"]:
    count+=1
    
    if count == 1:
        copy_tree(dir, train_path+r'\train_1')
    elif count==2:
        copy_tree(dir, train_path+r'\train_2')
    elif count==3:
        copy_tree(dir, train_path+r'\train_3')

break


In [17]:
# proportion we want for train, val_test and the test datasets
prop_train=0.60
prop_val_test=0.40
prop_test=0.50 

In [18]:
labels={'label1':708,'label2':1426,'label3':930}

In [19]:
# count=0
# for index, letter in enumerate(os.listdir(train_path)):
#     count+=1
#     folder_letter_source = os.path.join(train_path, letter)
#     folder_letter_destiny_val = os.path.join(val_path, str("val_{0}".format(str(count))))
#     #folder_letter_destiny_test = os.path.join(train_path, letter)
#     #print(folder_letter_destiny_val)

#     images_val = random.sample(os.listdir(folder_letter_source), k=prop_val_test*labels['label{0}'.format(str(count))])
    
    

In [20]:
# input images in VALIDATION

count=0
for index, letter in enumerate(os.listdir(train_path)):
    count+=1
    folder_letter_source = os.path.join(train_path, letter)
    folder_letter_destiny_val = os.path.join(val_path, str("val_{0}".format(str(count))))

    images_val = random.sample(os.listdir(folder_letter_source), k=round(prop_val_test*labels['label{0}'.format(str(count))]))
    count1=0
    
    for image in images_val:
        count1+=1
        if count1==1 and len(os.listdir(folder_letter_destiny_val)) != 0:
            print('break')
            for f in os.listdir(folder_letter_destiny_val):
                os.remove(os.path.join(folder_letter_destiny_val, f))
            
        src = os.path.join(folder_letter_source, image)
        dst = os.path.join(folder_letter_destiny_val, image)
        shutil.move(src, dst)

    # images_test = random.sample(os.listdir(folder_letter_source), k=images_for_test_val)
    # for image in images_test:
    #     src = os.path.join(folder_letter_source, image)
    #     dst = os.path.join(folder_letter_destiny_test, image)
    #     shutil.move(src, dst)


break
break
break


In [21]:
labels['label1']=round(labels['label1']*0.4)
labels['label2']=round(labels['label2']*0.4)
labels['label3']=round(labels['label3']*0.4)
labels

{'label1': 283, 'label2': 570, 'label3': 372}

In [22]:
# input images in TEST

count=0
for index, letter in enumerate(os.listdir(val_path)):
    count+=1
    folder_letter_source = os.path.join(val_path, letter)
    folder_letter_destiny_test = os.path.join(test_path, str("test_{0}".format(str(count))))

    images_val = random.sample(os.listdir(folder_letter_source), k=round(prop_test*labels['label{0}'.format(str(count))]))
    count1=0
    
    for image in images_val:
        count1+=1
        if count1==1 and len(os.listdir(folder_letter_destiny_test)) != 0:
            print('break')
            for f in os.listdir(folder_letter_destiny_test):
                os.remove(os.path.join(folder_letter_destiny_test, f))
            
        src = os.path.join(folder_letter_source, image)
        dst = os.path.join(folder_letter_destiny_test, image)
        shutil.move(src, dst)
        

break
break
break


In [6]:

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),  # resizes all images # qual é o tamanho ideal?
    batch_size=20,
    
    class_mode='categorical',
    color_mode='grayscale',
)

Found 1839 images belonging to 3 classes.
